In [1]:
%pylab inline
import sqlite3
import pandas
import tqdm
from sklearn import *
import ml_metrics
from xgboost import sklearn as xgblearn

Populating the interactive namespace from numpy and matplotlib


/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [2]:
def fit_mini_batch(reg, test=False, train_batch=None):
    # Fit three little estimators in each
    # round of boosting, all from the same
    # sample.
    reg.n_estimators += 4
    if train_batch is None:
        train_batch = np.random.randint(100)
    con = sqlite3.connect('/tmp/train_test_data.sqlite3')
    try:
        print('Fetching data: %d' % train_batch)
        data = pandas.read_sql('''
            SELECT * 
              FROM train_data 
             WHERE adjusted_demand is not null 
                   AND rand = ? AND week_num < 8''', con=con, params=[train_batch])
    finally:
        con.close()
    data = data.drop(['week_num', 'rand'], axis=1)
    X = data.drop('adjusted_demand', axis=1).as_matrix().copy(order='C')
    # See https://github.com/dmlc/xgboost/issues/446#issuecomment-135555130
    y = np.log(1+data.adjusted_demand.as_matrix().copy(order='C'))
    reg.fit(X, y)

    if test:
        test_batch = np.random.randint(100)
        con = sqlite3.connect('/tmp/train_test_data.sqlite3')
        try:
            data = pandas.read_sql('''
                SELECT * 
                  FROM test_data 
                 WHERE adjusted_demand is not null 
                       AND rand = ? AND week_num >= 8''', con=con, params=[test_batch])
        finally:
            con.close()

        data = data.drop(['week_num', 'rand'], axis=1)
        X = data.drop('adjusted_demand', axis=1).as_matrix().copy(order='C')
        pred = exp(reg.predict(X)) - 1
        print(ml_metrics.rmsle(data.adjusted_demand, pred))

In [3]:
reg = ensemble.GradientBoostingRegressor(
    warm_start=True,
    max_depth=11,
    verbose=0, 
    n_estimators=0)

reg = ensemble.ExtraTreesRegressor(
    warm_start=True,
    verbose=0, 
    n_jobs=-1,
    n_estimators=0)

In [ ]:
for i in range(200):
    fit_mini_batch(reg, i % 3 == 0, i % 100)

Fetching data: 0
0.622702913953
Fetching data: 1
Fetching data: 2
Fetching data: 3
0.55303572727
Fetching data: 4
Fetching data: 5
Fetching data: 6
0.544279092619
Fetching data: 7
Fetching data: 8
Fetching data: 9
0.542323045181
Fetching data: 10
Fetching data: 11
Fetching data: 12
0.539148382405
Fetching data: 13
Fetching data: 14
Fetching data: 15
0.537835313243
Fetching data: 16
Fetching data: 17
Fetching data: 18
0.536780169741
Fetching data: 19
Fetching data: 20
Fetching data: 21
0.534108935727
Fetching data: 22
Fetching data: 23
Fetching data: 24
0.534377574525
Fetching data: 25
Fetching data: 26
Fetching data: 27
0.534578613834
Fetching data: 28
Fetching data: 29
Fetching data: 30
0.533864573578
Fetching data: 31
Fetching data: 32
Fetching data: 33
0.533348243399
Fetching data: 34
Fetching data: 35
Fetching data: 36
0.532102327581
Fetching data: 37
Fetching data: 38
Fetching data: 39
0.535613510905
Fetching data: 40
Fetching data: 41
Fetching data: 42
0.533281960298
Fetching dat